In [15]:
"""
Text-based parser for ProteinNet Records.
"""

__author__ = "Mohammed AlQuraishi"
__copyright__ = "Copyright 2019, Harvard Medical School"
__license__ = "MIT"

#!/usr/bin/python

# imports
import sys
import re
import numpy as np

# Constants
NUM_DIMENSIONS = 3

# Functions for conversion from Mathematica protein files to TFRecords
_aa_dict = {'A': '0', 'C': '1', 'D': '2', 'E': '3', 'F': '4', 'G': '5', 'H': '6', 'I': '7', 'K': '8', 'L': '9', 'M': '10', 'N': '11', 'P': '12', 'Q': '13', 'R': '14', 'S': '15', 'T': '16', 'V': '17', 'W': '18', 'Y': '19'}
_dssp_dict = {'L': '0', 'H': '1', 'B': '2', 'E': '3', 'G': '4', 'I': '5', 'T': '6', 'S': '7'}
_mask_dict = {'-': '0', '+': '1'}


def letter_to_num(string, dict_):
    """ Convert string of letters to list of ints """
    patt = re.compile('[' + ''.join(dict_.keys()) + ']')
    num_string = patt.sub(lambda m: dict_[m.group(0)] + ' ', string)
    num = [int(i) for i in num_string.split()]
    return num

def read_record(file_, num_evo_entries):
    """ Read a Mathematica protein record from file and convert into dict. """
    
    dict_ = {}
    
            
    while True:
        next_line = file_.readline()
        if next_line == '[ID]' + '\n':
            id_ = file_.readline()[:-1]
            dict_.update({'id': id_})
        elif next_line == '[PRIMARY]' + '\n':
            primary = letter_to_num(file_.readline()[:-1], _aa_dict)
            dict_.update({'primary': primary})
        elif next_line == '[EVOLUTIONARY]' + '\n':
            evolutionary = []
            for residue in range(num_evo_entries): evolutionary.append([float(step) for step in file_.readline().split()])
            dict_.update({'evolutionary': evolutionary})
        elif next_line == '[SECONDARY]' + '\n':
            print(next_line)
            secondary = letter_to_num(file_.readline()[:-1], _dssp_dict)
            dict_.update({'secondary': secondary})
        elif next_line == '[TERTIARY]' + '\n':
            tertiary = []
            for axis in range(NUM_DIMENSIONS): tertiary.append([float(coord) for coord in file_.readline().split()])
            dict_.update({'tertiary': tertiary})
        elif next_line == '[MASK]' + '\n':
            mask = letter_to_num(file_.readline()[:-1], _mask_dict)
            dict_.update({'mask': mask})
        elif next_line == '\n':
            return dict_
        elif next_line == '':
            return None
        
        
    


In [16]:
def dict_to_protein_shape(dict_):
    protein = []
    amino_acid_f = []
    for i in range(len(dict_['primary'])):
        #primary structure
        prim_lab = [0] * 21
        index = dict_['primary'][i]
        prim_lab[index] = 1
    
        #secondary label
        sec_lab = [0] * 8
    
        #PSSM
        pssm = []
        for j in range(21):
            pssm.append(dict_['evolutionary'][j][i])
        
        amino_acid_f = prim_lab + sec_lab + pssm    
        protein.append(amino_acid_f)
        amino_acid_f = []

    while (len (protein) < 700):
        no_seq = [0] * 50
        protein.append(no_seq)
    
    #returns protein in shape (1,700,50)
    protein = np.array(protein)   
    return protein    

In [17]:
import gzip
def save_parti_dataset(proteins, caspName , index):
    
    dataset = np.zeros(( len(proteins), 700, 50))

    for i in range ( len(proteins) ):
        dataset[i] = proteins[i]
    
    f = gzip.GzipFile( caspName + "-" + str(index) + '.npy.gz', "w")
    np.save(f, dataset)
    f.close()
    print(dataset.shape)

In [18]:
# main. accepts two command-line arguments: input file and the number of entries in evo profiles, and outputs dicts to stdout

input_path =   "casp12/training_30"  #= sys.argv[1] 
num_evo_entries = 21 #int(sys.argv[2]) if len(sys.argv) == 3 else 20 # default number of evo entries

input_file = open(input_path, 'r')
   
proteins = [] #np.zeros((1,700,50)) 

i = 0
datIndex = 0
transkaya_conter = 0
while True:
    
    #progress bar
    if i % 1000 == 0:
        print(i)
        
    #save first partition of dataset - same size as transkaya        
    if transkaya_conter == 5278:
        print("saving dataset")
        save_parti_dataset(proteins, input_path, datIndex)
        proteins = []
        transkaya_conter = 0
        datIndex += 1
    
    #reading file
    dict_ = read_record(input_file, num_evo_entries)
    if dict_ is not None:
        protein = dict_to_protein_shape(dict_)
        
        #700 is max len for proteins
        if (protein.shape[0] == 700):
            proteins.append(protein)
            transkaya_conter += 1
            i += 1 
            #proteins = np.vstack((proteins,protein))
    else:
        input_file.close()
        break
    

#removing the first elemnt used for stacking    
#proteins = np.delete(proteins, 0)    

0
1000
2000
3000
4000
5000
saving dataset
(5278, 700, 50)
6000
7000
8000
9000
10000
saving dataset
(5278, 700, 50)
11000
12000
13000
14000
14000
15000
saving dataset
(5278, 700, 50)
16000
17000
18000
19000
20000
21000
saving dataset
(5278, 700, 50)
22000
23000
24000


In [ ]:
import mmap

#secondary label is not present in the files
with open('casp12/training_100', 'rb', 0) as file, \
     mmap.mmap(file.fileno(), 0, access=mmap.ACCESS_READ) as s:
    if s.find(b'SECONDARY') != -1:
        print('true')
    else: 
        print('false')

In [9]:
#finding the number of total proteins before creating numpy on disk
nr_windows = 0
for i in range(19):
    print(i)
    f = gzip.GzipFile( 'casp12/training_100-' + str(i) + 'window19Middle' +'.npy.gz', "r")
    dataset_part =  np.load(f)
    print(dataset_part.shape[0], " shape of dataset")
    nr_windows += dataset_part.shape[0]
    print("all togheter ", nr_windows)
    
    del dataset_part

0
1036506  shape of dataset
all togheter  1036506
1
1044591  shape of dataset
all togheter  2081097
2
1035235  shape of dataset
all togheter  3116332
3
1036940  shape of dataset
all togheter  4153272
4
1039488  shape of dataset
all togheter  5192760
5
1041722  shape of dataset
all togheter  6234482
6
1039770  shape of dataset
all togheter  7274252
7
1036662  shape of dataset
all togheter  8310914
8
1028068  shape of dataset
all togheter  9338982
9
1030480  shape of dataset
all togheter  10369462
10
1021110  shape of dataset
all togheter  11390572
11
1034139  shape of dataset
all togheter  12424711
12
1014951  shape of dataset
all togheter  13439662
13
1024966  shape of dataset
all togheter  14464628
14
1031661  shape of dataset
all togheter  15496289
15
1017105  shape of dataset
all togheter  16513394
16
1031984  shape of dataset
all togheter  17545378
17
1030908  shape of dataset
all togheter  18576286
18
1032898  shape of dataset
all togheter  19609184


In [14]:
filename = "casp12/proteinNet-all-windows-19-middle"
# 19609184
#all 19 partitions in 1 dataset
fp = np.memmap(filename, dtype='float64', mode='w+', shape=(19609184, 19, 50))

OSError: [Errno 28] No space left on device

In [ ]:
#adding all the information to big dataset
l = 0
r = 0
for i in range(19):
    
    f = gzip.GzipFile( 'casp12/training_100-' + str(i) + 'window19Middle' +'.npy.gz', "r")
    dataset_part =  np.load(f)
    print(dataset_part.shape)
    
    r += dataset_part.shape[0]
    print(l, " ", r)
    fp[l:r] =  dataset_part
    l = r

In [ ]:

fp.flush()
del fp

In [ ]:
fp = np.memmap(filename, dtype='float64', mode='r', shape=(1133502 * 2, 19, 50))

In [ ]:
0
1000
2000
3000
4000
5000
saving dataset
(5278, 700, 50)
6000
7000
8000
9000
10000
saving dataset
(5278, 700, 50)
11000
12000
13000
14000
15000
saving dataset
(5278, 700, 50)
16000
17000
18000
19000
20000
21000
saving dataset
(5278, 700, 50)
22000
23000
24000
25000
26000
saving dataset
(5278, 700, 50)
27000
28000
29000
30000
31000
saving dataset
(5278, 700, 50)
32000
33000
34000
35000
36000
saving dataset
(5278, 700, 50)
37000
38000
39000
40000
41000
41000
42000
saving dataset
(5278, 700, 50)
43000
44000
45000
46000
47000
saving dataset
(5278, 700, 50)
48000
49000
50000
51000
52000
saving dataset
(5278, 700, 50)
53000
54000
55000
56000
57000
58000
saving dataset
(5278, 700, 50)
59000
60000
61000
62000
63000
saving dataset
(5278, 700, 50)
64000
65000
66000
67000
68000
saving dataset
(5278, 700, 50)
69000
70000
71000
72000
73000
saving dataset
(5278, 700, 50)
74000
75000
76000
77000
78000
79000
saving dataset
(5278, 700, 50)
80000
81000
82000
83000
84000
saving dataset
(5278, 700, 50)
85000
86000
87000
88000
89000
saving dataset
(5278, 700, 50)
90000
91000
92000
93000
94000
95000
saving dataset
(5278, 700, 50)
96000
96000
97000
98000
99000
100000
saving dataset
(5278, 700, 50)
101000
102000